In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
#import matplotlib.pyplot as plt
import psycopg2

In [2]:
fact_table_cols = [('Individual_Key', 'Resolved', 'Fatal', 'Special_Measure_Key', 'PHU_Key', 'Weather_Key', 'Onset_Date_Key', 'Reported_Date_Key', 'Test_Date_Key', 'Specimen_Date_Key')]
individual_cols = [('surrogate_key', 'individual_id', 'date', 'age_group', 'gender', 'cause_of_infection', 'outbreak_related', 'reporting_phu_city')]
fact_table = []
individual_table = []

def connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        return psycopg2.connect(
            host="www.eecs.uottawa.ca",
            database="group_21",
            user="***",
            password="***",
            port="15432"
        )
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)


connection = connect()
print('***Connection established***')

cur = connection.cursor()
        
# execute a statement
cur.execute('SELECT * FROM fact_table_v2')
rows = cur.fetchall()
fact_table = rows
# print(fact_table[:10])


print("\n\n")


cur.execute('SELECT * FROM individuals')
rows = cur.fetchall()
individual_table = rows
# print(individual_table[:10])

cur.close()
connection.close()
print('***Connection terminated***')

Connecting to the PostgreSQL database...
***Connection established***



***Connection terminated***


In [3]:
fact_df = pd.DataFrame(fact_table, columns=['Individual_Key', 'Resolved', 'Fatal', 'Special_Measure_Key', 'PHU_Key', 'Weather_Key', 'Onset_Date_Key', 'Reported_Date_Key', 'Test_Date_Key', 'Specimen_Date_Key'])
fact_df.head()

,Individual_Key,Resolved,Fatal,Special_Measure_Key,PHU_Key,Weather_Key,Onset_Date_Key,Reported_Date_Key,Test_Date_Key,Specimen_Date_Key
0,250,yes,no,11,3895,123,27,32,32,29
1,322,yes,no,11,3895,110,16,19,19,18
2,326,no,yes,11,3895,96,1,5,28,1
3,347,yes,no,11,3895,107,9,16,10,9
4,692,yes,no,11,3895,116,29,25,25,24


In [4]:
# creating instance of labelencoder
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
fact_df['Special_Measure_Key_Cat'] = labelencoder.fit_transform(fact_df['Special_Measure_Key'])
fact_df.head()

,Individual_Key,Resolved,Fatal,Special_Measure_Key,PHU_Key,Weather_Key,Onset_Date_Key,Reported_Date_Key,Test_Date_Key,Specimen_Date_Key,Special_Measure_Key_Cat
0,250,yes,no,11,3895,123,27,32,32,29,6
1,322,yes,no,11,3895,110,16,19,19,18,6
2,326,no,yes,11,3895,96,1,5,28,1,6
3,347,yes,no,11,3895,107,9,16,10,9,6
4,692,yes,no,11,3895,116,29,25,25,24,6


In [5]:
# creating instance of one-hot-encoder

enc = OneHotEncoder(handle_unknown='ignore')


In [6]:
# passing bridge-types-cat column (label encoded values of bridge_types)

enc_df = pd.DataFrame(enc.fit_transform(fact_df[['Special_Measure_Key_Cat']]).toarray())


In [7]:
# merge with main df bridge_df on key values

fact_df = fact_df.join(enc_df)
fact_df.rename(columns={6: "lockdown", 10: "stage 3 extended to toronto", 2: "Restart", 1: "Gatineau-Ottawa Travel Restriction", 3: "Parks Re-opened"},inplace = True)
fact_df=fact_df.drop(['Specimen_Date_Key','Onset_Date_Key', 'Reported_Date_Key', 'Test_Date_Key','Resolved','PHU_Key','Weather_Key','Fatal',4,5], axis=1)
fact_df.head()


,Individual_Key,Special_Measure_Key,Special_Measure_Key_Cat,0,Gatineau-Ottawa Travel Restriction,Restart,Parks Re-opened,lockdown
0,250,11,6,0.0,0.0,0.0,0.0,1.0
1,322,11,6,0.0,0.0,0.0,0.0,1.0
2,326,11,6,0.0,0.0,0.0,0.0,1.0
3,347,11,6,0.0,0.0,0.0,0.0,1.0
4,692,11,6,0.0,0.0,0.0,0.0,1.0
